In [1]:
import numpy as np
import pandas as pd

#Create a DataFrame from companies.csv
companies = pd.read_csv("companies.csv",sep = "\\t", encoding='ANSI', engine = 'python')
companies.head()

FileNotFoundError: [Errno 2] No such file or directory: 'companies.csv'

In [ ]:
#change permalink column to lowercase
companies['permalink'] = pd.DataFrame(map(lambda x: x.lower(), companies.permalink))

# The number of unique companies in "companies"
companies.nunique(axis = 0)

In [ ]:
#Create a DataFrame from rounds2.csv
rounds2 = pd.read_csv("rounds2.csv", encoding='ANSI')
rounds2.head()

In [ ]:
#change company_permalink column to lowercase
rounds2['company_permalink'] = pd.DataFrame(map(lambda x: x.lower(), rounds2.company_permalink))

# The number of unique companies in "rounds2"
rounds2.nunique(axis = 0)

In [ ]:
# Renaming the column "permalink" to "company_permalink" to make it unique_id while merging with rounds2
companies.rename(columns={'permalink':'company_permalink'}, inplace=True)
companies.head()

In [ ]:
# Outer merge of companies to rounds2 to find any company only present in any of the DataFrame
master_frame1 = pd.merge(rounds2, companies, on='company_permalink', how='outer', indicator='Exist')
diff_df = master_frame1.loc[master_frame1['Exist'] != 'both']
diff_df

In [ ]:
# Found a company whose permalink had an extra semicolon in companies dataframe from that in rounds2. 

inRounds2 = diff_df.loc[114849, 'company_permalink']
inCompanies = diff_df.loc[114949, 'company_permalink']

In [ ]:
# Fixing the error in comapnies DataFrame
companies.loc[companies.company_permalink == inCompanies, 'company_permalink'] = inRounds2

In [ ]:
# Final merging of comapnies with rounds2
master_frame = pd.merge(rounds2, companies, on='company_permalink', how='inner')
master_frame = master_frame.drop(['funding_round_code','homepage_url','state_code','region','founded_at'], axis=1)
master_frame.head()

1) Calculate the most representative value of the investment amount for each of the four funding types (venture, angel, seed, and private equity).

2) Based on the most representative investment amount calculated above, which investment type do you think is the most suitable for Spark Funds?

In [ ]:
# Remove rows with where investment value is NaN
master_frame.dropna(subset=['raised_amount_usd'], inplace = True)

In [ ]:
# Grouping by funding_round_type column
df_by_fundingtype = master_frame.groupby('funding_round_type')
df_by_fundingtype

In [ ]:
# Finding the average investments in Venture, angel, seed, private_equity fund types
# This is to find the most suitable fund type to invest with capital of 5 mill - 15 mill USD
average_investments = pd.DataFrame(df_by_fundingtype['raised_amount_usd'].mean())
average_investments.sort_values('raised_amount_usd', ascending = False)

average_investments

In [ ]:
list_of_funding_types = (average_investments.reset_index())['funding_round_type'].tolist()
list_of_funding_types

In [ ]:
import matplotlib.pyplot as plt


funding_type_counts = list(map(lambda c : (master_frame.funding_round_type == c).sum(), list_of_funding_types))
funding_type_dict = {'funding_types':list_of_funding_types, 'total count': funding_type_counts}

allfundtypescount = pd.DataFrame(funding_type_dict)
allfundtypescount = allfundtypescount.set_index('funding_types')
focusfindingtypes = allfundtypescount.loc[['venture','seed','private_equity'], :]

allfundtypescount = allfundtypescount.drop(['venture','seed','private_equity'])
focusfindingtypes
allfundtypescount.loc['Others'] = allfundtypescount['total count'].sum()
allfundtypescount
focusfindingtypes.loc['Others'] = allfundtypescount.loc['Others']
focusfindingtypes = focusfindingtypes.reset_index()
focusfindingtypes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


plt.figure(figsize=(10, 6))

# subplot 1: Funding type preferred by investors globally
fundstype = focusfindingtypes['funding_types']
count = focusfindingtypes['total count']
explode = (0.1, 0, 0, 0) 
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]
plt.pie(count, labels=fundstype, explode=explode,
autopct='%1.1f%%', shadow=True, startangle=360)
plt.title("Funding type preferred by investors globally")

plt.show()




1) Spark Funds wants to see the top nine countries which have received the highest total funding (across ALL sectors for the    chosen investment type)

2) For the chosen investment type, make a data frame named top9 with the top nine countries (based on the total investment amount each country has received)

In [ ]:
#Finding top 9 countries for venture investment
pi_table = master_frame.pivot_table(values = 'raised_amount_usd', 
                      index = 'country_code', 
                      columns = 'funding_round_type', 
                      aggfunc = 'sum')
pi_table.dropna(subset=['venture'], inplace = True)
df_venture = pd.DataFrame(pi_table.venture)
allcountries = df_venture.sort_values('venture', ascending = False)
top9 = allcountries.loc['USA':'JPN', :].reset_index()
# top9 = top9.reset_index()
top9

1) Extract the primary sector of each category list from the category_list column

2) Use the mapping file 'mapping.csv' to map each primary sector to one of the eight main sectors (Note that ‘Others’ is also considered one of the main sectors)

In [ ]:
mapping = pd.read_csv("mapping.csv", encoding='ANSI')

In [ ]:
master_frame['category_list'] = master_frame['category_list'].fillna("")
master_frame['primary_sector'] = master_frame['category_list'].apply(lambda x:x.split("|")[0])
#master_frame(master_frame.category_list == 0)

In [ ]:
sector = pd.DataFrame(mapping.category_list)
sector

In [ ]:
sector['main_sector'] = mapping.apply(lambda x: mapping.columns[x==1].values[0], axis = 1)
sector.rename(columns={'category_list':'primary_sector'}, inplace=True)

In [ ]:
master_frame = pd.merge(master_frame, sector, on = 'primary_sector', how = 'inner')
master_frame

1) Create three separate data frames for each of the three countries containing the observations of funding type FT falling within the 5-15 million USD range. The three data frames should contain:

2) All the columns of the master_frame along with the primary sector and the main sector

3) The total number (or count) of investments for each main sector in a separate column

4) The total amount invested in each main sector in a separate column

In [ ]:
df_USA = master_frame.loc[(master_frame['country_code'] == 'USA') & (master_frame['funding_round_type'] == 'venture') & (master_frame['raised_amount_usd'] >= 5000000) & (master_frame['raised_amount_usd'] <= 15000000)]
df_USA

In [ ]:
main_sector_list = ["Entertainment", "Manufacturing", "Others", "News, Search and Messaging", "Cleantech / Semiconductors", "Health", "Social, Finance, Analytics, Advertising", "Automotive & Sports"]


In [ ]:
main_sector_counts_usa = list(map(lambda c : (df_USA.main_sector == c).sum(), main_sector_list))
main_sec_dic_usa = {'main_sector':main_sector_list, 'total count': main_sector_counts_usa}

In [ ]:
tobemergedsectorcount_usa = pd.DataFrame(main_sec_dic_usa)
tobemergedsectorcount_usa

tobeplottedcount_usa = tobemergedsectorcount_usa.set_index('main_sector')
tobeplottedcount_usa = tobeplottedcount_usa.sort_values('total count', ascending = False)
tobeplottedcount_usa = tobeplottedcount_usa.loc[['Others','Cleantech / Semiconductors','Social, Finance, Analytics, Advertising'],:]
tobeplottedcount_usa

In [ ]:
tobemergedsectorcount_usa['total count'].sum()

In [ ]:
df_USA = pd.merge(df_USA, tobemergedsectorcount_usa, on = "main_sector")
df_USA

In [ ]:
totalamounttobemerged_usa = df_USA.pivot_table(values = 'raised_amount_usd', 
                      index = 'main_sector',
                      aggfunc = 'sum')
totalamounttobemerged_usa.rename(columns={'raised_amount_usd':'total investment/main_sector'}, inplace=True)

In [ ]:
df_USA = pd.merge(df_USA, totalamounttobemerged_usa, on = "main_sector")
df_USA

In [ ]:
totalamounttobemerged_usa['total investment/main_sector'].sum()

In [ ]:
comapanywithtopsectorinvestment_usa = df_USA[df_USA.main_sector == "Others"]
comapanywithtopsectorinvestment_usa.pivot_table(values = 'raised_amount_usd', 
                      index = 'name',
                      aggfunc = 'sum').sort_values('raised_amount_usd', ascending = False).astype(str)

In [ ]:
comapanywithsecondtopsectorinvestment_usa = df_USA[df_USA.main_sector == "Cleantech / Semiconductors"]
comapanywithsecondtopsectorinvestment_usa.pivot_table(values = 'raised_amount_usd', 
                      index = 'name',
                      aggfunc = 'sum').sort_values('raised_amount_usd', ascending = False).astype(str)

In [ ]:
df_GBR = master_frame.loc[(master_frame['country_code'] == 'GBR') & (master_frame['funding_round_type'] == 'venture') & (master_frame['raised_amount_usd'] >= 5000000) & (master_frame['raised_amount_usd'] <= 15000000)]
df_GBR

In [ ]:
main_sector_counts_gbr = list(map(lambda c : (df_GBR.main_sector == c).sum(), main_sector_list))
main_sec_dic_gbr = {'main_sector':main_sector_list, 'total count': main_sector_counts_gbr}

In [ ]:
tobemergedsectorcount_gbr = pd.DataFrame(main_sec_dic_gbr)
tobeplottedcount_gbr = tobemergedsectorcount_gbr.set_index('main_sector')
tobeplottedcount_gbr = tobeplottedcount_gbr.sort_values('total count', ascending = False)
tobeplottedcount_gbr = tobeplottedcount_gbr.loc[['Others','Cleantech / Semiconductors','Social, Finance, Analytics, Advertising'],:]
tobeplottedcount_gbr

In [ ]:
tobemergedsectorcount_gbr['total count'].sum()

In [ ]:
df_GBR = pd.merge(df_GBR, tobemergedsectorcount_gbr, on = "main_sector")
df_GBR

In [ ]:
totalamounttobemerged_gbr = df_GBR.pivot_table(values = 'raised_amount_usd', 
                      index = 'main_sector',
                      aggfunc = 'sum')
totalamounttobemerged_gbr.rename(columns={'raised_amount_usd':'total investment/main_sector'}, inplace=True)
totalamounttobemerged_gbr.astype(str)

In [ ]:
df_GBR = pd.merge(df_GBR, totalamounttobemerged_gbr, on = "main_sector")
df_GBR

In [ ]:
totalamounttobemerged_gbr['total investment/main_sector'].sum()

In [ ]:
comapanywithtopsectorinvestment_gbr = df_GBR[df_GBR.main_sector == "Others"]
comapanywithtopsectorinvestment_gbr.pivot_table(values = 'raised_amount_usd', 
                      index = 'name',
                      aggfunc = 'sum').sort_values('raised_amount_usd', ascending = False).astype(str)

In [ ]:
comapanywithsecondtopsectorinvestment_gbr = df_GBR[df_GBR.main_sector == "Cleantech / Semiconductors"]
comapanywithsecondtopsectorinvestment_gbr.pivot_table(values = 'raised_amount_usd', 
                      index = 'name',
                      aggfunc = 'sum').sort_values('raised_amount_usd', ascending = False).astype(str)

In [ ]:
df_IND = master_frame.loc[(master_frame['country_code'] == 'IND') & (master_frame['funding_round_type'] == 'venture') & (master_frame['raised_amount_usd'] >= 5000000) & (master_frame['raised_amount_usd'] <= 15000000)]
df_IND

In [ ]:
main_sector_counts_ind = list(map(lambda c : (df_IND.main_sector == c).sum(), main_sector_list))
main_sec_dic_ind = {'main_sector':main_sector_list, 'total count': main_sector_counts_ind}

In [ ]:
tobemergedsectorcount_ind = pd.DataFrame(main_sec_dic_ind)
tobeplottedcount_ind = tobemergedsectorcount_ind.set_index('main_sector')
tobeplottedcount_ind = tobeplottedcount_ind.sort_values('total count', ascending = False)
tobeplottedcount_ind = tobeplottedcount_ind.loc[['Others','News, Search and Messaging','Entertainment'],:]
tobeplottedcount_ind

In [ ]:
tobemergedsectorcount_ind['total count'].sum()

In [ ]:
df_IND = pd.merge(df_IND, tobemergedsectorcount_ind, on = "main_sector")
df_IND

In [ ]:
totalamounttobemerged_ind = df_IND.pivot_table(values = 'raised_amount_usd', 
                      index = 'main_sector',
                      aggfunc = 'sum')
totalamounttobemerged_ind.rename(columns={'raised_amount_usd':'total investment/main_sector'}, inplace=True)
totalamounttobemerged_ind.astype(str)

In [ ]:
df_IND = pd.merge(df_IND, totalamounttobemerged_ind, on = "main_sector")
df_IND

In [ ]:
totalamounttobemerged_ind['total investment/main_sector'].sum()

In [ ]:
comapanywithtopsectorinvestment_ind = df_IND[df_IND.main_sector == "Others"]
comapanywithtopsectorinvestment_ind.pivot_table(values = 'raised_amount_usd', 
                      index = 'name',
                      aggfunc = 'sum').sort_values('raised_amount_usd', ascending = False).astype(str)

In [ ]:
comapanywithsecondtopsectorinvestment_ind = df_IND[df_IND.main_sector == "News, Search and Messaging"]
comapanywithsecondtopsectorinvestment_ind.pivot_table(values = 'raised_amount_usd', 
                      index = 'name',
                      aggfunc = 'sum').sort_values('raised_amount_usd', ascending = False).astype(str)

In [ ]:
focused_investement = average_investments.loc[['private_equity','venture','seed'],:]
focused_investement = focused_investement.reset_index()
focused_investement

In [ ]:
focusfindingtypes = focusfindingtypes.drop(3)
focusfindingtypes

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 10))

# subplot 1: Share in number of investments among Venture, Private Equity and Seed funds
plt.subplot(1, 2, 1)
fundstype = focusfindingtypes['funding_types']
count = focusfindingtypes['total count']
explode = (0.1, 0, 0) 
colors = ["#1f77b4", "#ff7f0e", "#2ca02c"]
plt.pie(count, labels=fundstype, explode=explode,
autopct='%1.1f%%', shadow=True, startangle=360)
plt.title("Share in number of investments among Venture, Private Equity and Seed funds.")

# subplot 2: Investment average in Private equity, venture and seed
plt.subplot(1, 2, 2)
sns.boxplot(x='funding_round_type', y='raised_amount_usd', data=focused_investement)
plt.yscale('log')
plt.title("Average Investments")

plt.show()




In [ ]:
plt.figure(figsize=(12, 10))
top9
sns.barplot(x='country_code', y='venture', data=top9)
plt.title("Total Investments / Country")

plt.show()

In [ ]:
tobeplottedcount_usa['Country'] = "USA"
tobeplottedcount_gbr['Country'] = "GBR"
tobeplottedcount_ind['Country'] = "IND"
con = pd.concat([tobeplottedcount_usa, tobeplottedcount_gbr, tobeplottedcount_ind])
con = con.reset_index()
con

In [ ]:
plt.figure(figsize=(12, 10))
sns.catplot(x="Country", y="total count", hue="main_sector", data=con,
                height=6, kind="bar", palette="muted")
plt.title("Number of investments in the top 3 sectors of the top 3 countries")

plt.show()